# EP - PSI3471 (2024 - $1^o$ semestre)

|Nome do aluno                     |NUSP    |E-mail USP       |
|----------------------------------|--------|-----------------|
|Gustavo Henrique da Silva Amaral  |12551686|gustavo.amaral7@usp.br|
|Thiago da Rocha Calomino Gonçalves|12554647|thcalomino@usp.br|

Prof. Hae Yong Kim

Link: [Enunciado do EP 2024](http://www.lps.usp.br/hae/psi3471/ep1-2024/ep.pdf)

## Instalação de dependências

In [ ]:
import cv2
import numpy as np
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from collections import Counter

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Importação das imagens do Kaggle

In [ ]:
# Importar e configurar kaggle
!pip3 install -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"thiagocalomino","key":"295190f2a38c974aa9e5613554a8c124"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

# Baixar o dataset de pistache
!kaggle datasets download -d muratkokludataset/pistachio-image-dataset

# Descompactar o dataset
!unzip -q pistachio-image-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/muratkokludataset/pistachio-image-dataset
License(s): CC-BY-NC-SA-4.0
 94% 25.0M/26.7M [00:00<00:00, 86.0MB/s]
100% 26.7M/26.7M [00:00<00:00, 75.3MB/s]


# Redução e alinhamento horizontal das imagens

In [ ]:
def find_center_and_orientation(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Escala de cinza
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)  # Converte em imagem binária

    moments = cv2.moments(binary)  # Momentos da imagem binária
    cen_x = moments['m10'] / moments['m00']  # Coord. X do centro de massa
    cen_y = moments['m01'] / moments['m00']  # Coord. Y do centro de massa
    theta = 0.5 * np.arctan2(2 * moments['mu11'], moments['mu20'] - moments['mu02'])  # Calcula a orientação

    return (cen_x, cen_y, theta)  # Retorna o centro de massa e a orientação

def reduce_and_align_image(image, size):
    reduced_image = cv2.resize(image, (size, size), interpolation=cv2.INTER_AREA)  # Redução para dimensão dada
    cen_x, cen_y, theta = find_center_and_orientation(reduced_image)  # Encontra o centro e a orientação da imagem reduzida
    height, width = reduced_image.shape[:2]
    M = cv2.getRotationMatrix2D((cen_x, cen_y), np.degrees(theta), 1)  # Calcula a matriz de rotação para alinhar a imagem horizontalmente
    aligned_image = cv2.warpAffine(reduced_image, M, (width, height), flags=cv2.INTER_LINEAR)  # Aplica a rotação para alinhar a imagem

    return reduced_image, aligned_image  # Retorna a imagem reduzida e alinhada

def process_images(input_dir, output_dir, size=128):
    reduced_dir = os.path.join(output_dir, 'R')  # Diretório para salvar as imagens reduzidas
    aligned_dir = os.path.join(output_dir, 'A')  # Diretório para salvar as imagens alinhadas

    if not os.path.exists(reduced_dir):
        os.makedirs(reduced_dir)  # Cria o diretório para imagens reduzidas se não existir
    if not os.path.exists(aligned_dir):
        os.makedirs(aligned_dir)  # Cria o diretório para imagens alinhadas se não existir

    images = glob.glob(f"{input_dir}/*.jpg")  # Lista todas as imagens .jpg no diretório de entrada
    print(f"Encontradas {len(images)} imagens.")

    for image_path in images:
        image = cv2.imread(image_path)  # Lê a imagem

        reduced_image, aligned_image = reduce_and_align_image(image, size)  # Processa a imagem para reduzir e alinhar
        base_name = os.path.basename(image_path)

        reduced_image_path = os.path.join(reduced_dir, f"R_{base_name}")  # Caminho para salvar a imagem reduzida
        aligned_image_path = os.path.join(aligned_dir, f"A_{base_name}")  # Caminho para salvar a imagem alinhada

        cv2.imwrite(reduced_image_path, reduced_image)  # Salva a imagem reduzida
        cv2.imwrite(aligned_image_path, aligned_image)  # Salva a imagem alinhada

In [ ]:
# Redução e alinhamento das imagens de pistache
process_images('Pistachio_Image_Dataset/Pistachio_Image_Dataset/Kirmizi_Pistachio', 'processed_images', size=128)
process_images('Pistachio_Image_Dataset/Pistachio_Image_Dataset/Siirt_Pistachio', 'processed_images', size=128)

Encontradas 1232 imagens.
Encontradas 916 imagens.


# Função para carregar os dados

In [ ]:
def load_dataset(directory):
    data = []
    labels = []
    print(f"Carregando imagens do diretório: {directory}")

    for filename in os.listdir(directory):  # Itera sobre todos os arquivos no diretório
        if filename.endswith(".jpg"):  # Verifica se o arquivo é uma imagem .jpg

            # Verifica o nome do grão e define o seu respectivo rótulo
            if 'kirmizi' in filename.lower():
                label = 'kirmizi'
            elif 'siirt' in filename.lower():
                label = 'siirt'

            image_path = os.path.join(directory, filename)
            image = load_img(image_path, target_size=(128, 128))  # Carrega a imagem e redimensiona para 128 x 128
            image = img_to_array(image)  # Converte para NumPy Array
            data.append(image)
            labels.append(label)

    print(f"Total de imagens carregadas: {len(data)}")

    data = np.array(data, dtype="float") / 255.0  # Normaliza os valores dos pixels para o intervalo [0, 1]
    le = LabelEncoder()
    labels = le.fit_transform(labels)  # Converte os rótulos para valores inteiros
    return data, labels, le

# Implementação por SVM

In [ ]:
def classify_images(data, labels):
    class_distribution = Counter(labels)  # Conta a distribuição das classes
    print(f"Distribuição das classes: {class_distribution}")

    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.5, stratify=labels)  # Divide os dados em treino e teste

    # Escalonamento dos dados
    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX.reshape(trainX.shape[0], -1))  # Ajusta e transforma os dados de treino
    testX = scaler.transform(testX.reshape(testX.shape[0], -1))  # Transforma os dados de teste usando os parâmetros calculados nos dados de treino

    model = SVC(kernel='linear', C=1)  # Define o modelo SVM com kernel linear e C=1
    model.fit(trainX, trainY)  # Treina o modelo
    predictions = model.predict(testX)  # Faz previsões nos dados de teste
    accuracy = accuracy_score(testY, predictions)  # Calcula a acurácia das previsões
    return accuracy

In [ ]:
# Carregar e classificar imagens reduzidas
dataR, labelsR, leR = load_dataset('processed_images/R')
print(f"Classes disponíveis: {leR.classes_}")
reduced_accuracy = classify_images(dataR, labelsR)
print(f"Taxa de acerto (imagens reduzidas): {100*reduced_accuracy} %")

Carregando imagens do diretório: processed_images/R
Total de imagens carregadas: 2148
Classes disponíveis: ['kirmizi' 'siirt']
Distribuição das classes: Counter({0: 1232, 1: 916})
Taxa de acerto (imagens reduzidas): 83.9851024208566 %


In [ ]:
# Carregar e classificar imagens alinhadas
dataA, labelsA, leA = load_dataset('processed_images/A')
print(f"Classes disponíveis: {leA.classes_}")
aligned_accuracy = classify_images(dataA, labelsA)
print(f"Taxa de acerto (imagens alinhadas): {100 * aligned_accuracy} %")

Carregando imagens do diretório: processed_images/A
Total de imagens carregadas: 2148
Classes disponíveis: ['kirmizi' 'siirt']
Distribuição das classes: Counter({0: 1232, 1: 916})
Taxa de acerto (imagens alinhadas): 82.68156424581005 %


# Implementação por CNN

In [ ]:
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # Primeira camada convolucional
        MaxPooling2D((2, 2)),  # Primeira camada de pooling
        Conv2D(64, (3, 3), activation='relu'),  # Segunda camada convolucional
        MaxPooling2D((2, 2)),  # Segunda camada de pooling
        Conv2D(128, (3, 3), activation='relu'),  # Terceira camada convolucional
        MaxPooling2D((2, 2)),  # Terceira camada de pooling
        Flatten(),  # Achata a saída das camadas convolucionais para uma única dimensão
        Dense(256, activation='relu'),  # Primeira camada totalmente conectada com 256 neurônios
        Dense(128, activation='relu'),  # Segunda camada totalmente conectada com 128 neurônios
        Dense(2, activation='softmax')  # Camada de saída com 2 neurônios para as classes 'kirmizi' e 'siirt'
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compila o modelo
    return model

def classify_images_with_cnn(data, labels):
    class_distribution = Counter(labels)  # Conta a distribuição das classes
    print(f"Distribuição das classes: {class_distribution}")

    trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.5, stratify=labels)  # Divide os dados em treino e teste

    model = create_cnn_model(input_shape=(128, 128, 3)) # Cria o modelo de CNN
    model.fit(trainX, trainY, batch_size=32, epochs=10, verbose=1)  # Treina o modelo
    loss, accuracy = model.evaluate(testX, testY)  # Avalia o modelo nos dados de teste
    return accuracy

In [ ]:
# Classificar imagens reduzidas com CNN
print(f"Classes disponíveis: {leR.classes_}")
reduced_accuracy = classify_images_with_cnn(dataR, labelsR)
print(f"Taxa de acerto CNN (imagens reduzidas): {100 * reduced_accuracy} %")

Classes disponíveis: ['kirmizi' 'siirt']
Distribuição das classes: Counter({0: 1232, 1: 916})
Epoch 1/10
34/34 [==============================] - 7s 33ms/step - loss: 0.5469 - accuracy: 0.7393
Epoch 2/10
34/34 [==============================] - 1s 18ms/step - loss: 0.3990 - accuracy: 0.8128
Epoch 3/10
34/34 [==============================] - 1s 20ms/step - loss: 0.3868 - accuracy: 0.8352
Epoch 4/10
34/34 [==============================] - 1s 21ms/step - loss: 0.3390 - accuracy: 0.8566
Epoch 5/10
34/34 [==============================] - 1s 21ms/step - loss: 0.3128 - accuracy: 0.8687
Epoch 6/10
34/34 [==============================] - 1s 20ms/step - loss: 0.3076 - accuracy: 0.8706
Epoch 7/10
34/34 [==============================] - 1s 18ms/step - loss: 0.2860 - accuracy: 0.8855
Epoch 8/10
34/34 [==============================] - 1s 18ms/step - loss: 0.2361 - accuracy: 0.9032
Epoch 9/10
34/34 [==============================] - 1s 18ms/step - loss: 0.2058 - accuracy: 0.9125
Epoch 10/10
34/

In [ ]:
# Classificar imagens alinhadas com CNN
print(f"Classes disponíveis: {leA.classes_}")
aligned_accuracy = classify_images_with_cnn(dataA, labelsA)
print(f"Taxa de acerto CNN (imagens alinhadas): {100 * aligned_accuracy} %")

Classes disponíveis: ['kirmizi' 'siirt']
Distribuição das classes: Counter({0: 1232, 1: 916})
Epoch 1/10
34/34 [==============================] - 2s 21ms/step - loss: 0.4930 - accuracy: 0.7542
Epoch 2/10
34/34 [==============================] - 1s 18ms/step - loss: 0.3944 - accuracy: 0.8110
Epoch 3/10
34/34 [==============================] - 1s 19ms/step - loss: 0.3412 - accuracy: 0.8464
Epoch 4/10
34/34 [==============================] - 1s 22ms/step - loss: 0.2984 - accuracy: 0.8696
Epoch 5/10
34/34 [==============================] - 1s 22ms/step - loss: 0.2915 - accuracy: 0.8669
Epoch 6/10
34/34 [==============================] - 1s 22ms/step - loss: 0.2440 - accuracy: 0.8939
Epoch 7/10
34/34 [==============================] - 1s 19ms/step - loss: 0.2398 - accuracy: 0.8929
Epoch 8/10
34/34 [==============================] - 1s 18ms/step - loss: 0.2014 - accuracy: 0.9209
Epoch 9/10
34/34 [==============================] - 1s 18ms/step - loss: 0.1519 - accuracy: 0.9330
Epoch 10/10
34/